In [1]:
from pyspark.sql import SparkSession, Row
from pyspark import SparkContext
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType

In [8]:
# Initialize SparkSession
spark = SparkSession.builder.appName("NA values").getOrCreate()
sc = SparkContext.getOrCreate()

In [9]:
rdd = sc.parallelize([Row(server_name='101 server', cpu_utilization=85, session_count=80), \
                         Row(server_name='101 server', cpu_utilization=80, session_count=90), \
                         Row(server_name='102 server', cpu_utilization=85, session_count=40), \
                         Row(server_name='103 server', cpu_utilization=70, session_count=80), \
                         Row(server_name='104 server', cpu_utilization=60, session_count=80)])

In [11]:
# Convert RDD to DataFrame
df = spark.createDataFrame(rdd)

In [12]:
df.show()

+-----------+---------------+-------------+
|server_name|cpu_utilization|session_count|
+-----------+---------------+-------------+
| 101 server|             85|           80|
| 101 server|             80|           90|
| 102 server|             85|           40|
| 103 server|             70|           80|
| 104 server|             60|           80|
+-----------+---------------+-------------+



In [13]:
df_na = df.withColumn('na_col', lit(None).cast(StringType()))

In [14]:
df_na.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 server|             85|           80|  NULL|
| 101 server|             80|           90|  NULL|
| 102 server|             85|           40|  NULL|
| 103 server|             70|           80|  NULL|
| 104 server|             60|           80|  NULL|
+-----------+---------------+-------------+------+



In [15]:
df_na.fillna('A').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 server|             85|           80|     A|
| 101 server|             80|           90|     A|
| 102 server|             85|           40|     A|
| 103 server|             70|           80|     A|
| 104 server|             60|           80|     A|
+-----------+---------------+-------------+------+



In [16]:
df2 = df_na.fillna('A').union(df_na)

In [17]:
df2.show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 server|             85|           80|     A|
| 101 server|             80|           90|     A|
| 102 server|             85|           40|     A|
| 103 server|             70|           80|     A|
| 104 server|             60|           80|     A|
| 101 server|             85|           80|  NULL|
| 101 server|             80|           90|  NULL|
| 102 server|             85|           40|  NULL|
| 103 server|             70|           80|  NULL|
| 104 server|             60|           80|  NULL|
+-----------+---------------+-------------+------+



In [18]:
df2.na.drop().show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 server|             85|           80|     A|
| 101 server|             80|           90|     A|
| 102 server|             85|           40|     A|
| 103 server|             70|           80|     A|
| 104 server|             60|           80|     A|
+-----------+---------------+-------------+------+



In [19]:
df2.createOrReplaceTempView('na_table')

In [20]:
spark.sql('SELECT * FROM na_table').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 server|             85|           80|     A|
| 101 server|             80|           90|     A|
| 102 server|             85|           40|     A|
| 103 server|             70|           80|     A|
| 104 server|             60|           80|     A|
| 101 server|             85|           80|  NULL|
| 101 server|             80|           90|  NULL|
| 102 server|             85|           40|  NULL|
| 103 server|             70|           80|  NULL|
| 104 server|             60|           80|  NULL|
+-----------+---------------+-------------+------+



In [22]:
spark.sql('SELECT * FROM na_table WHERE na_col IS NULL').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 server|             85|           80|  NULL|
| 101 server|             80|           90|  NULL|
| 102 server|             85|           40|  NULL|
| 103 server|             70|           80|  NULL|
| 104 server|             60|           80|  NULL|
+-----------+---------------+-------------+------+



In [23]:
spark.sql('SELECT * FROM na_table WHERE na_col IS NOT NULL').show()

+-----------+---------------+-------------+------+
|server_name|cpu_utilization|session_count|na_col|
+-----------+---------------+-------------+------+
| 101 server|             85|           80|     A|
| 101 server|             80|           90|     A|
| 102 server|             85|           40|     A|
| 103 server|             70|           80|     A|
| 104 server|             60|           80|     A|
+-----------+---------------+-------------+------+

